# 准备阶段

## 加载依赖的块和库

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn import preprocessing
from scipy.stats import skew, boxcox
import os

from pyecharts import Bar
import mpld3
from mpld3 import plugins

%matplotlib inline

# 探索性数据分析(EDA)及可视化

In [ ]:
# 定义异常值范围
na_values=['','NULL','NA','null','na','Na','-9999','Infinity','NaN']
xpath='D:\\\xc8\xd5\xb3\xa3\xb7\xd6\xce\xf6\xb1\xa8\xb8\xe6\\\xb3\xac\xd0\xa1\xb6\xee\xb7\xd6\xce\xf6\\'
# 读取数据集
data_set_name=pd.read_table(xpath+'data\\data_set_name.txt',header=None,sep='\t',na_values = na_values)
data_set=pd.read_table(xpath+'data\\data_set.txt',header=None,sep='\t',na_values = na_values,names=data_set_name[0],index_col=None)

# 查看数据集信息
print('---------------------------------------数据预览：---------------------------------------------')
print(data_set.head())

print('---------------------------------------数据集基本信息：---------------------------------------')
print(data_set.info())

print('---------------------------------------数据统计信息：-----------------------------------------')
print(data_set.describe())

## 数据处理

### 聚合分类变量

In [ ]:
# 申请城市city_idx聚合为1线城市，2线城市，3线城市及其他
data_set.loc[data_set.city_idx==1.0,'city_index_cut']='1_city'
data_set.loc[data_set.city_idx==1.5,'city_index_cut']='2_city'
data_set.loc[data_set.city_idx==2.0,'city_index_cut']='3_city'
data_set.loc[(data_set.city_idx!=1.0) & (data_set.city_idx!=1.5) & (data_set.city_idx!=2.0),'city_index_cut']='4_city'

# 构建新变量，并按照索引排序
data_set['city_index_cut'].value_counts(dropna=False).sort_index()

### 离散化数值型变量

In [ ]:
# age
age_bins=[0,18,22,28,35,45,55,np.inf]
age_group_names=range(1,len(age_bins))
cats=pd.cut(list(data_set['age']),age_bins,right = False,labels=age_group_names) # right=False 表示左闭口右开
data_set['age_cut']=list(cats)

# 同盾和百度多头
duotou_vars=['jiedaiscore','bidnormalscoreexp3v2','prcidnormalscoreexp3v2','phonenormalscoreexp3v2']
duotou_bins=[0,20,40,60,80,100]
duotou_group_name=range(1,len(duotou_bins))
for col in duotou_vars:
    cats=pd.cut(list(data_set[col]),duotou_bins,right = False,labels=duotou_group_name)
    data_set[str(col)+'_'+'cut']=list(cats)


# 手机入网时长 mobileOperatorRegDate
mobile_bins=[0,10,20,30,40,np.inf]
mobile_group_name=range(1,len(mobile_bins))
cats=pd.cut(list(data_set['mobileoperatorregdate']),mobile_bins,right=False,labels=mobile_group_name)
data_set['mobileoperatorregdate_cut']=list(cats)


# 征信
zx_vars=['zxaccountcnt','zxhouseloancnt']
zx_bins=[-np.inf,1,5,10,np.inf]
zx_group_name=range(1,len(zx_bins))
for col in zx_vars:
    cats=pd.cut(list(data_set[col]),zx_bins,right=False,labels=zx_group_name)
    data_set[str(col)+'_'+'cut']=list(cats)
    

# preaficoscorev5
prea_bins=[0,480,520,550,600,650,np.inf]
pre_group_name=range(1,len(prea_bins))
cats=pd.cut(list(data_set['preaficoscorev5']),prea_bins,right=False,labels=pre_group_name)
data_set['preaficoscorev5_cut']=list(cats)



In [ ]:
# 概览
data_set[['preaficoscorev5','preaficoscorev5_cut','zxaccountcnt','zxaccountcnt_cut','age','age_cut']].head()

### 构建逻辑变量

In [ ]:
# 判断是否授信
data_set['apply_bid']=1

In [ ]:
# 判断用户是否用信：due_days大于等于0，表示用户用信
action=[]
for i in range(len(data_set)):
    if data_set['due_days'][i]>=0:
        action.append(1)
    else :
        action.append(0)

data_set['action']=action

In [ ]:
#  判断是否逾期用户
odu_bid=[]
for i in range(len(data_set)):
    if data_set['due_days'][i]>0:
        odu_bid.append(1)
    else :
        odu_bid.append(0)

data_set['odu_bid']=odu_bid

In [ ]:
# 判断是否是超小额用户
petty_bid=[]
for i in range(len(data_set)):
    if data_set['initialamount'][i] == 50000:
        petty_bid.append(1)
    else :
        petty_bid.append(0)

data_set['petty_bid']=petty_bid

In [ ]:
# 判断是否是用信超小额用户
action_petty_bid=[]
for i in range(len(data_set)):
    if data_set['initialamount'][i] == 50000 and data_set['due_days'][i]>=0:
        action_petty_bid.append(1)
    else :
        action_petty_bid.append(0)

data_set['action_petty_bid']=action_petty_bid

In [ ]:
# 将客群分成三类，即A，B，C
data_set.loc[data_set.is_white==1,'white_petty_bid']='A'
data_set.loc[(data_set.is_white==0)&(data_set.petty_bid==0),'white_petty_bid']='B'
data_set.loc[(data_set.is_white==0)&(data_set.petty_bid==1),'white_petty_bid']='C'

### 基本数据概览

In [ ]:
white_petty_grouped = data_set.groupby('white_petty_bid')
# 迭代groupby对象
for group, frame in white_petty_grouped:
    apply_sum = frame['bid'].count()
    action_sum = frame['action'].sum()
    odu_sum = frame['odu_bid'].sum()
    ratio_action=round(frame['action'].sum()*1.0/frame['bid'].count()*100,2)
    ratio_odu=round(frame['odu_bid'].sum()*1.0/frame['action'].sum()*100,2)
    print('{}群体授信人数：{}，用信人数：{}，逾期人数{} , 用信率(%)：{}，逾期率(%)：{}'.format(group, apply_sum, action_sum, odu_sum , ratio_action,ratio_odu))

结论1：
* 整体授信群体一共1083380人，用信581076人，逾期19076人。用信率：53.6%，逾期率：3.2%
* 白名单群体用信率和逾期率跟大盘持平
* 非白群体中，超小额群体用信意愿最低，且逾期率也最高，明显高于整体非白群体

## 连续变量统计信息表

* 对于分类型变量，考虑不同组合下，不同群体的分布变化

### (白名单，超小额)客群分布

In [ ]:
# 表格：超小额用户群体用信人数和逾期人数
grouped = data_set.groupby(['is_white','petty_bid'])
for group, frame in grouped:
    print('（白名单，超小额）为{}的申请用户个数：{}'.format(group, len(frame)))

df=grouped.agg({'apply_bid':np.sum,'action': np.sum ,'odu_bid':np.sum})

# 三类客群申请，用信及逾期分布
df['授信人数']=df['apply_bid']
df['用信人数']=df['action']
df['用信率']=(df['action']*1.0/df['apply_bid']).apply(lambda x: format(x, '.1%'))
df['逾期人数']=df['odu_bid']
df['逾期率']=(df['odu_bid']*1.0/df['action']).apply(lambda x: format(x, '.1%'))
df.drop(['action','odu_bid','apply_bid'],axis=1)

**存在的问题：**

* 超小额群体的授信，用信和逾期情况主要集中在非白群体中
* 考虑将上述四象限群体合并为白名单用户(A客群)，非白非超小额群体(B客群)，非白超小额群体(C客群)

### 三类客群整体分布

In [ ]:
# 表格：超小额用户群体用信人数和逾期人数
grouped = data_set.groupby(['white_petty_bid'])
for group, frame in grouped:
    print('{}客群的申请用户个数：{}'.format(group, len(frame)))

df=grouped.agg({'apply_bid':np.sum,'action': np.sum ,'odu_bid':np.sum})

# 三类客群申请，用信及逾期分布
df['授信人数']=df['apply_bid']
df['用信人数']=df['action']
df['用信率']=(df['action']*1.0/df['apply_bid']).apply(lambda x: format(x, '.1%'))
df['逾期人数']=df['odu_bid']
df['逾期率']=(df['odu_bid']*1.0/df['action']).apply(lambda x: format(x, '.1%'))
df.drop(['action','odu_bid','apply_bid'],axis=1)

In [ ]:
# 数据覆盖问题
need_cols=['white_petty_bid','blacklisttype','tongdunblacklevel','jiedaiscore','mobileoperatorregdate','zxhouseloancnt','zxaccountcnt','bidnormalscoreexp3v2','phonenormalscoreexp3v2','prcidnormalscoreexp3v2']
data_set_0=data_set[need_cols]
df_A=data_set_0[data_set_0.white_petty_bid=='A']
df_B=data_set_0[data_set_0.white_petty_bid=='B']
df_C=data_set_0[data_set_0.white_petty_bid=='C']

# 计算覆盖率
def get_freq_prcent(df):
    cover_dt=df.isnull().describe().T
    cover_dt['freq_prcent']=(cover_dt['freq']/cover_dt['count']).apply(lambda x : format(x ,'.1%'))
    cover_dt_a=cover_dt.drop(['unique','top'],axis=1)
    return cover_dt_a

In [ ]:
# 分别三类群体的覆盖度并左右拼接数据，横向对比
df_A_1=get_freq_prcent(df_A).rename(columns={'count':'cnt_a','freq':'freq_a','freq_prcent':'freq_prcent_a'})
df_B_1=get_freq_prcent(df_B).rename(columns={'count':'cnt_b','freq':'freq_b','freq_prcent':'freq_prcent_b'})
df_C_1=get_freq_prcent(df_C).rename(columns={'count':'cnt_c','freq':'freq_c','freq_prcent':'freq_prcent_c'})

print ('\n A，B，C三类人群对应特征字段非空比例：')
pd.concat([df_A_1,df_B_1,df_C_1],axis=1)[['freq_prcent_a','freq_prcent_b','freq_prcent_c']]

** 定位问题 : **

* C类人群至少是一群**征信覆盖比较低**的人
* C类客群用信率比较低，即无太大用信意愿，而对于用信的用户而言，逾期率又普遍偏高。

### 三类客群统计信息

In [ ]:
# stat cols：连续值
stat_cols_float=['white_petty_bid','age','jiedaiscore','zxhouseloancnt','zxaccountcnt','bidnormalscoreexp3v2','prcidnormalscoreexp3v2','phonenormalscoreexp3v2','preaficoscorev5']
data_set_float=data_set[stat_cols_float]

# 子集：连续变量
df_stat=data_set_float.groupby(['white_petty_bid'])
df_result=df_stat.describe().transpose()[['A','C']]
df_result

### **结论：**

* 年龄普遍偏低
* bidnormalscoreexp3v2分值偏低且分布相对集中
* jiedaiscore分值偏高，整体分布跟白名单用户类似
* phonenormalscoreexp3v2和prcidnormalscoreexp3v2分布几乎无差异
* preaficoscorev5分布相对较集中，且整体分值偏低；
* zxaccountcnt分布整体偏低且较集中；zxhouseloancnt无显著区分

### 补充：相关性分析

#### 变量相关性

In [ ]:
# # 查看变量间的相关性
corr_data=data_set[['white_petty_bid','jiedaiscore','bidnormalscoreexp3v2','prcidnormalscoreexp3v2','preaficoscorev5','zxhouseloancnt','zxaccountcnt','age','phonenormalscoreexp3v2']]
corr_data.columns = ['white', 'jie','bind','prcid','prea','zxh','zxa','age','phone']
sns.heatmap(corr_data.corr())

#### 高斯密度曲线

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# 筛选非白群体
df=data_set.sample(n=20000)
sns_data=df[['white_petty_bid','jiedaiscore','bidnormalscoreexp3v2','preaficoscorev5','zxhouseloancnt','zxaccountcnt','age']]
sns.pairplot(sns_data, hue='white_petty_bid', diag_kind='kde')

补充：

* kde曲线可以避免因bins手工设置不合理导致的数据分布异常的情况发生

## 分类数据分布及占比

* 分析white_petty_bid群体（细分为A，B，C）的用信和逾期指标在不同 **X特征** 下的分布情况


In [ ]:
# 离散数据求占比
import copy
def ratio(input_data):
    output_data=copy.deepcopy(input_data)
    n=output_data.shape[0]
    m=input_data.shape[1]-1
    for col in list(output_data.columns):
        for i in range(n):
            if col=='dt':
                output_data['dt'][i]=input_data.iloc[i][m]
            else :
                output_data[col][i]=round(100.0*input_data[col][i]/input_data.iloc[:,m][i],2)
    return output_data

### 年龄分段

* 年龄分段：超小额群体年龄多集中在2区间内，即18~22岁

In [ ]:
# 不同目标群体在不同年龄区间的用信和逾期分布
df1=data_set.pivot_table(values=['apply_bid'], index='white_petty_bid', columns=['age_cut'], aggfunc=np.sum, margins=True)
pd.concat([df1['apply_bid'],ratio(df1['apply_bid'])],axis=1)

### jiedai分段

* jiedaiscore分段：超小额群体年龄多集中在4区间内，即60~80分

In [ ]:
# 不同目标群体在不同年龄区间的用信和逾期分布
df2=data_set.pivot_table(values=['apply_bid'], index='white_petty_bid', columns=['jiedaiscore_cut'], aggfunc=np.sum, margins=True)
pd.concat([df2['apply_bid'],ratio(df2['apply_bid'])],axis=1)

### 同盾黑名单等级

* tongdunblacklevel：G3等级黑名单人群占比相对较高

In [ ]:
# 不同目标群体在不同年龄区间的用信和逾期分布
df3=data_set.pivot_table(values=['apply_bid'], index='white_petty_bid', columns=['tongdunblacklevel'], aggfunc=np.sum, margins=True)
pd.concat([df3['apply_bid'],ratio(df3['apply_bid'])],axis=1)

### 城市等级

* 城市等级：4线及以上城市较多

In [ ]:
# 不同目标群体在不同年龄区间的用信和逾期分布
df4=data_set.pivot_table(values=['apply_bid'], index='white_petty_bid', columns=['city_index_cut'], aggfunc=np.sum, margins=True)
pd.concat([df4['apply_bid'],ratio(df4['apply_bid'])],axis=1)

### 教育等级

* education：多为无学历信息

In [ ]:
# 不同目标群体在不同年龄区间的用信和逾期分布
df5=data_set.pivot_table(values=['apply_bid'], index='white_petty_bid', columns=['education'], aggfunc=np.sum, margins=True)
pd.concat([df5['apply_bid'],ratio(df5['apply_bid'])],axis=1)

### preacustseg

* preacustseg：UNK标识占大多数

In [ ]:
# 不同目标群体在不同年龄区间的用信和逾期分布
df6=data_set.pivot_table(values=['apply_bid'], index='white_petty_bid', columns=['preacustseg'], aggfunc=np.sum, margins=True)
pd.concat([df6['apply_bid'],ratio(df6['apply_bid'])],axis=1)

### **结论：**

* 年龄分段：超小额群体年龄多集中在2区间内，即18~22岁
* jiedaiscore分段：超小额群体年龄多集中在4区间内，即60~80分
* tongdunblacklevel：G3等级黑名单人群占比相对较高
* education：多为无学历信息
* 城市等级：4线及以上城市较多
* preacustseg：UNK标识占大多数

### 补充：作图

#### 箱线图

In [ ]:
# 箱图分布
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

a = sns.boxplot(x='white_petty_bid', y='age', data=data_set, ax=axs[0][0])
# axs[0][0].set_yscale('log')

b = sns.boxplot(x='white_petty_bid', y='jiedaiscore', data=data_set, ax=axs[0][1])
# axs[0][1].set(ylim=(0, 0.5e8))

c = sns.boxplot(x='white_petty_bid', y='preaficoscorev5', data=data_set, ax=axs[1][0])
# axs[1][0].set(ylim=(0, 3e7))

d = sns.boxplot(x='white_petty_bid', y='zxaccountcnt', data=data_set, ax=axs[1][1])

plt.show()


#### 散点图

In [ ]:
g=sns.FacetGrid(data_set,hue="white_petty_bid",col="education",margin_titles=True)
g.map(plt.scatter,"initialamount","age",alpha=0.7)
g.add_legend()

## 趋势性分析

In [ ]:
# 首先构建时间变量
from datetime import datetime, date, time
from time import strftime, localtime

apply_time=data_set['apply_time'].astype(str)
# 分别添加日期维度和周维度
apply_day=[]
apply_week=[]
for i in apply_time:
    a_1=datetime.strptime(i,"%Y%m%d%H%M%S").strftime("%m-%d")
    w_1=datetime.strptime(i,"%Y%m%d%H%M%S").strftime("%Y-%W")
    apply_day.append(a_1)
    apply_week.append(w_1)

data_set['apply_dt']=list(apply_day)
data_set['apply_day_w']=list(apply_week)
# data_set['apply_dt']=data_set['apply_dt'].astype(object)

In [ ]:
# 不同目标群体逾期人数时间分布
line_petty_df=data_set.pivot_table(values=['apply_bid'], index='apply_dt', columns='white_petty_bid', aggfunc=[np.sum])
# line_petty_df.plot()

# 时间趋势
fig, ax = plt.subplots(figsize=(12, 8))
labels = ['A','B','C']
lines = ax.plot( line_petty_df.values, lw=2)
interactive_legend = plugins.InteractiveLegendPlugin(lines, labels)
plugins.connect(fig, interactive_legend)

mpld3.display()

In [ ]:
# 箱图分布
fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# 透视表
df=pd.pivot_table(data_set[data_set.white_petty_bid!='C'],index=["age_cut"], columns=["apply_day_w"], values=["due_days"],aggfunc=np.sum)
a=sns.heatmap(df['due_days'],annot=False,fmt="0.1f",ax=axs[0][0])

# # 透视表
df=pd.pivot_table(data_set[data_set.white_petty_bid=='C'],index=["age_cut"], columns=["apply_day_w"], values=["due_days"],aggfunc=np.sum)
b=sns.heatmap(df['due_days'],annot=False,fmt="0.1f",ax=axs[0][1])


# 透视表
df=pd.pivot_table(data_set[data_set.white_petty_bid!='C'],index=["tongdunblacklevel"], columns=["apply_day_w"], values=["due_days"],aggfunc=np.sum)
c=sns.heatmap(df['due_days'],annot=False,fmt="0.1f",ax=axs[1][0])

# # 透视表
df=pd.pivot_table(data_set[data_set.white_petty_bid=='C'],index=["tongdunblacklevel"], columns=["apply_day_w"], values=["due_days"],aggfunc=np.sum)
d=sns.heatmap(df['due_days'],annot=False,fmt="0.1f",ax=axs[1][1])


plt.show()

结论:
* 从热力图可以看出，18~22岁人群虽然差，但是从逾期上来看，整体age都不好

# 超小额群体行为特征

## 准备数据

In [ ]:
# 超小额用户群体
petty_data_set=data_set[data_set.white_petty_bid=='C']

In [ ]:
# 表格：超小额用户群体用信人数和逾期人数
grouped = petty_data_set.groupby(['white_petty_bid'])
for group, frame in grouped:
    print('{}客群的申请用户个数：{}'.format(group, len(frame)))

df=grouped.agg({'apply_bid':np.sum,'action': np.sum ,'odu_bid':np.sum})

# 三类客群申请，用信及逾期分布
df['授信人数']=df['apply_bid']
df['用信人数']=df['action']
df['用信率']=(df['action']*1.0/df['apply_bid']).apply(lambda x: format(x, '.1%'))
df['逾期人数']=df['odu_bid']
df['逾期率']=(df['odu_bid']*1.0/df['action']).apply(lambda x: format(x, '.1%'))
df.drop(['action','odu_bid','apply_bid'],axis=1)

## 用信用户是否逾期变量特征

### 用户是否逾期散点图

In [ ]:
# 筛选用信用户
petty_action_dt=petty_data_set[petty_data_set.action==1]

In [ ]:
# 筛选非白群体
petty_action_dt.loc[petty_action_dt.odu_bid==0,'odu_type']='odu_0'
petty_action_dt.loc[petty_action_dt.odu_bid==1,'odu_type']='odu_1'
sns_data=petty_action_dt[['odu_type','jiedaiscore','bidnormalscoreexp3v2','preaficoscorev5','zxhouseloancnt','zxaccountcnt','age']]

# 散点图 palette="Set2"
sns.pairplot(sns_data, hue='odu_type', diag_kind='kde',size=2,markers="+")

解读：

* 直观上看，简单的变量组合很难完全区分出超小额逾期用户和未逾期用户。散点图分布重叠度比较高
* 同时，从变量的KDE曲线上也比较难以区分

In [ ]:
# 筛选特征
def get_vars_copula(df,col,row,bid_cnt,lower_ratio,upper_ratio):
    df_1=petty_action_dt.pivot_table(values=['action','odu_bid'], index=col, columns=row, aggfunc={"action":len,"odu_bid":np.sum})
    df_2=df_1['odu_bid'][df_1.action>=bid_cnt]/df_1['action']
    return df_2[(df_2>=lower_ratio)&(df_2<upper_ratio)]

cut_vars=[x for x in petty_action_dt.columns if x.endswith('cut')]
append_vars=['preacustseg','tongdunblacklevel','blacklisttype','education']
index_vars=cut_vars+append_vars
col_vars=index_vars

# 用信人数大于30人，逾期率大于20%人群组合特征
# 如果全是Null值，则不输出
for i in index_vars:
    for j in col_vars:
        if i != j:
            a=get_vars_copula(petty_action_dt,i,j,30,0.2,1.1)
            m=a.shape[0];n=a.shape[1];p=a.isnull().sum().sum()
            if p!=m*n:
                print a

解读：

* mobileoperatorregdate_cut字段区分度较高
* city_index_cut：2线城市反倒差？

### 组合热力图

In [ ]:
# 筛选用信用户
petty_action_dt=petty_data_set[petty_data_set.action==1]

In [ ]:
# 箱图分布
fig, axs = plt.subplots(2, 2, figsize=(15, 15))
# threshhold=0.05

# 年龄
df1=petty_action_dt.pivot_table(values=['action','odu_bid'], index='mobileoperatorregdate_cut', columns='education', aggfunc={"action":len,"odu_bid":np.sum}, margins=True)
a1=df1['odu_bid']/df1['action']
h1=sns.heatmap(a1.T,annot=True,fmt="0.2f",ax=axs[0][0])

df1=petty_action_dt.pivot_table(values=['action','odu_bid'], index='mobileoperatorregdate_cut', columns='education', aggfunc={"action":len,"odu_bid":np.sum}, margins=True)
h2=sns.heatmap(df1['action'].T,annot=True,fmt="0.2f",ax=axs[0][1])


# 年龄
df1=petty_action_dt.pivot_table(values=['action','odu_bid'], index='blacklisttype', columns='city_index_cut', aggfunc={"action":len,"odu_bid":np.sum}, margins=True)
a1=df1['odu_bid']/df1['action']
h1=sns.heatmap(a1.T,annot=True,fmt="0.2f",ax=axs[1][0])

df1=petty_action_dt.pivot_table(values=['action','odu_bid'], index='blacklisttype', columns='city_index_cut', aggfunc={"action":len,"odu_bid":np.sum}, margins=True)
h2=sns.heatmap(df1['action'].T,annot=True,fmt="0.2f",ax=axs[1][1])


plt.show()


解读：

* 城市等级上之所以表现出二线城市高于三四线城市主要原因在于，三四线城市样本偏少

### 坏用户特征

* 手机入网时间不足一年，且无学历用户，整体逾期率在20%以上

In [ ]:
print 'A类群体中坏用户人数：{}；占比:{:.2%}'.format(data_set[(data_set.white_petty_bid =='A')&(data_set.mobileoperatorregdate_cut <=2)&(data_set.education==-1)].shape[0],data_set[(data_set.white_petty_bid =='A')&(data_set.mobileoperatorregdate_cut <=2)&(data_set.education==-1)].shape[0]*1.0/data_set.shape[0])
print 'B类群体中坏用户人数：{}；占比:{:.2%}'.format(data_set[(data_set.white_petty_bid =='B')&(data_set.mobileoperatorregdate_cut <=2)&(data_set.education==-1)].shape[0],data_set[(data_set.white_petty_bid =='B')&(data_set.mobileoperatorregdate_cut <=2)&(data_set.education==-1)].shape[0]*1.0/data_set.shape[0])
print 'C类群体中坏用户人数：{}；占比:{:.2%}'.format(data_set[(data_set.white_petty_bid =='C')&(data_set.mobileoperatorregdate_cut <=2)&(data_set.education==-1)].shape[0],data_set[(data_set.white_petty_bid =='C')&(data_set.mobileoperatorregdate_cut <=2)&(data_set.education==-1)].shape[0]*1.0/data_set.shape[0])
print '\n如果完全拒绝该类群体，平均每天拒绝用户人数增加{}人'.format((17569+4212+2010)/40)


#### 特征细分

In [ ]:
# 组合手机入网时间和学历特征
petty_action_dt.loc[petty_action_dt.education==-1,'edu']='no_edu'
petty_action_dt.loc[petty_action_dt.education!=-1,'edu']='edu'
petty_action_dt.loc[petty_action_dt.mobileoperatorregdate_cut<=2,'mobile_tag']='1_y'
petty_action_dt.loc[petty_action_dt.mobileoperatorregdate_cut>2,'mobile_tag']='2_y'
petty_action_dt['edu_tag_mobile']=petty_action_dt['edu']+'_'+petty_action_dt['mobile_tag']

In [ ]:
# 筛选特征
def get_vars_copula(df,col,row,bid_cnt,lower_ratio,upper_ratio):
    df_1=petty_action_dt.pivot_table(values=['action','odu_bid'], index=col, columns=row, aggfunc={"action":len,"odu_bid":np.sum})
    df_2=df_1['odu_bid'][df_1.action>=bid_cnt]/df_1['action']
    return df_2[(df_2>=lower_ratio)&(df_2<upper_ratio)]

cut_vars=[x for x in petty_action_dt.columns if x.endswith('cut')]
append_vars=['preacustseg','tongdunblacklevel','blacklisttype','edu_tag_mobile']
index_vars=cut_vars+append_vars
col_vars=['edu_tag_mobile']

# 用信人数大于30人，逾期率大于20%人群组合特征
# 如果全是Null值，则不输出
for i in index_vars:
    for j in col_vars:
        if i != j:
            a=get_vars_copula(petty_action_dt,i,j,30,0.2,1.1)
            m=a.shape[0];n=a.shape[1];p=a.isnull().sum().sum()
            if p!=m*n:
                print a

解读：

* 黑名单等级为G3的群体质量更差

In [ ]:
# 筛选特征
def get_vars_copula(df,col,row,bid_cnt,lower_ratio,upper_ratio):
    df_1=petty_action_dt.pivot_table(values=['action','odu_bid'], index=col, columns=row, aggfunc={"action":len,"odu_bid":np.sum})
    df_2=df_1['odu_bid'][df_1.action>=bid_cnt]/df_1['action']
    return df_2[(df_2>=lower_ratio)&(df_2<upper_ratio)]

cut_vars=[x for x in petty_action_dt.columns if x.endswith('cut')]
append_vars=['preacustseg','tongdunblacklevel','blacklisttype','edu_tag_mobile']
index_vars=cut_vars+append_vars
col_vars=index_vars

# 用信人数大于30人，逾期率大于20%人群组合特征
# 如果全是Null值，则不输出
for i in index_vars:
    for j in col_vars:
        if i != j:
            a=get_vars_copula(petty_action_dt,i,j,200,0,0.05)
            m=a.shape[0];n=a.shape[1];p=a.isnull().sum().sum()
            if p!=m*n:
                print a

好用户：
* bidnormalscoreexp3v2_cut<=1 & edu_2_y
* edu_2_y，mobile对应2年及以上
* 多头分比较低的C类客群也不错

# 建模

## 准备数据

In [ ]:
#加载所需库
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# 超小额用户群体
petty_data_set=data_set[data_set.white_petty_bid=='C']

In [ ]:
# 构建label
# 用信逾期和用信未逾期
petty_data_set.loc[(petty_data_set.action==1)&(petty_data_set.odu_bid==1),'label']="A"
petty_data_set.loc[(petty_data_set.action==1)&(petty_data_set.odu_bid==0),'label']="B"

In [ ]:
petty_data_set['label'].value_counts()

In [ ]:
# 数据集扩充和抽样
data_1=pd.concat([petty_data_set[petty_data_set.label=="A"]]*5)
data_2=petty_data_set[petty_data_set.label=="B"]


# 合并数据
data_set_new=pd.concat([data_1,data_2],ignore_index=True)

In [ ]:
# 筛选入模变量
need_cols=['age','preaficoscorev5','zxaccountcnt','bidnormalscoreexp3v2','jiedaiscore']
df_x=data_set_new[need_cols]
# 缺失值填充
df_x=df_x.fillna(0)

In [ ]:
# 数据归一化
model_data_x=df_x.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))   
# 筛选label
model_data_y=data_set_new['label']

In [ ]:
# 合并数据集.axis=1表示横向合并数据
model_data=pd.concat([model_data_x,model_data_y],axis=1)

In [ ]:
# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(model_data_x, model_data_y, test_size=0.3, random_state=0)
print('数据集样本数：{}，\n 训练集样本数：{}，\n 测试集样本数：{}'.format(len(model_data_x), len(X_train), len(X_test)))

## 模型建立及选择

In [ ]:
# 筛选最优的K值
k_range = range(1, 10)
acc_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    acc_scores.append(knn.score(X_test, y_test))


# k和acc对应值 : dict是无序的，怎么从无序的字典中找出最大项呢
acc_scores_dict=dict(zip(k_range,acc_scores))

from collections import Counter
# 找出value值最大项对应的key和value
Counter(acc_scores_dict).most_common(3)

In [ ]:
# 建立模型
knn = KNeighborsClassifier(n_neighbors=3)
# 训练模型
knn.fit(X_train, y_train)
# 测试模型/预测
y_pred = knn.predict(X_test)

# 概率预测输出每个样本属于某一类的概率
# 输出的结果是array。其中p[i][j] 是通过概率 kNN 判断 X[i] 属于第 j 类的概率
y_prob=knn.predict_proba(X_test)

# 两种计算精度的方法
acc = accuracy_score(y_test, y_pred)

print('acc_ratio:{:.2%}'.format(acc))

In [ ]:
# 忽略警告信息
import warnings
warnings.filterwarnings("ignore")

result_data=X_test
result_data['y_test']=y_test
result_data['y_pred']=y_pred
result_data.reset_index(drop=True).head(100)